In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv(r'house_pred.csv')

Preprocessing the train data

In [ ]:
for i in (data.columns):
    p=[]
    s=[]
    for j in data[i] :
       if j in p:
         k=p.index(j)
         s[k]+=1
       else:
           p.append(j)
           s.append(1)
    for j in s:
      if(j>=1200):
        print(p)
        print(s)
        print(i)
        data=data.drop(i,axis=1)

In [ ]:
matrix=data.corr()
x=matrix[(matrix["SalePrice"]<0.1)&(matrix["SalePrice"]>-0.1)]["SalePrice"]
j=list(x.index)
for i in j:
    data=data.drop(i,axis=1)

In [8]:
def change(x):
  if x=='Reg':
    pass
  else:
    x='IR'
  return x
lot_shape=data['LotShape']
lot_shape=lot_shape.apply(change)
data['LotShape']=lot_shape

In [31]:
data["LotFrontage"].fillna(0,inplace=True)

In [4]:
data=data.drop(["MsZoning","LotConfig","Neighborhood",'HouseStyle',"OverallQual","YearRemodAdd","RoofStyle","Exterior1st","Exterior2nd","MasVnrType","MasVnrArea",'ExterQual','Foundation','BsmtQual','BsmtExposure','BsmtFinSF1','BsmtFinType1','BsmtUnfSF','HeatingQC','GrLivArea','BsmtFullBath','FullBath','HalfBath','KitchenQual','TotRmsAbvGrd','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageCars','WoodDeckSF','OpenPorchSF','Fence','SaleCondition'],axis=1)

In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotFrontage   1460 non-null   float64
 1   LotArea       1460 non-null   int64  
 2   LotShape      1460 non-null   object 
 3   YearBuilt     1460 non-null   int64  
 4   TotalBsmtSF   1460 non-null   int64  
 5   1stFlrSF      1460 non-null   int64  
 6   2ndFlrSF      1460 non-null   int64  
 7   BedroomAbvGr  1460 non-null   int64  
 8   Fireplaces    1460 non-null   int64  
 9   GarageArea    1460 non-null   int64  
 10  SalePrice     1460 non-null   int64  
dtypes: float64(1), int64(9), object(1)
memory usage: 125.6+ KB


Encoding the LotShape column

In [61]:
shape=pd.get_dummies(data["LotShape"])
data=data.join(shape)
data=data.drop("LotShape",axis=1)
data.head()

,LotFrontage,LotArea,YearBuilt,TotalBsmtSF,1stFlrSF,2ndFlrSF,BedroomAbvGr,Fireplaces,GarageArea,SalePrice,IR,Reg
0,65.0,8450,2003,856,856,854,3,0,548,208500,0,1
1,80.0,9600,1976,1262,1262,0,3,1,460,181500,0,1
2,68.0,11250,2001,920,920,866,3,1,608,223500,1,0
3,60.0,9550,1915,756,961,756,3,1,642,140000,1,0
4,84.0,14260,2000,1145,1145,1053,4,1,836,250000,1,0


In [ ]:
data.columns

In [ ]:
col=['LotFrontage', 'LotArea', 'YearBuilt', 'TotalBsmtSF', '1stFlrSF','2ndFlrSF', 'BedroomAbvGr', 'Fireplaces', 'GarageArea', 'IR', 'Reg', 'SalePrice']
data=data[col]
data.head()

Splitting the data

In [64]:
x=data.iloc[:,:-1].values
y=data.iloc[:,-1].values

In [68]:
x=pd.DataFrame(x)
x.columns=['LotFrontage', 'LotArea', 'YearBuilt', 'TotalBsmtSF', '1stFlrSF','2ndFlrSF', 'BedroomAbvGr', 'Fireplaces', 'GarageArea', 'IR', 'Reg']

In [80]:
x=x.drop("IR",axis=1)
x=x.values
x=pd.DataFrame(x)
x.columns=['LotFrontage', 'LotArea', 'YearBuilt', 'TotalBsmtSF', '1stFlrSF','2ndFlrSF', 'BedroomAbvGr', 'Fireplaces', 'GarageArea', 'Reg', 'Intercept']

Checking the VIF

In [82]:
x['intercept']=1
vif = pd.DataFrame()
vif['variable']= x.columns
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif['vif'] = [variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
vif

In [85]:
x=x.drop("Intercept",axis=1)

Implementing Multiple Linear Regression

In [86]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x,y)

LinearRegression()

In [ ]:
reg.intercept_, reg.coef_

Testing the data

In [98]:
test=pd.read_csv("test.csv")

In [ ]:
test.info()

Preprocessing the test data

In [100]:
req_col=['LotFrontage','LotArea','LotShape','YearBuilt','TotalBsmtSF','1stFlrSF','2ndFlrSF','BedroomAbvGr','Fireplaces','GarageArea']
for i in test.columns:
  if i not in req_col:
    test=test.drop(i,axis=1)

In [104]:
def change(x):
  if x=='Reg':
    pass
  else:
    x='IR'
  return x
lot_shape=test['LotShape']
lot_shape=lot_shape.apply(change)
test['LotShape']=lot_shape

In [115]:
shape=pd.get_dummies(test["LotShape"])
test=test.join(shape)
test=test.drop(["LotShape","IR"],axis=1)
test.head()
test["LotFrontage"].fillna(0,inplace=True)

,LotFrontage,LotArea,YearBuilt,TotalBsmtSF,1stFlrSF,2ndFlrSF,BedroomAbvGr,Fireplaces,GarageArea,Reg
0,78.0,10140,1975,1056,1074,0,3,0,495,1
1,90.0,14684,1990,2158,2196,0,3,1,701,0
2,NaN,8900,1966,1056,1056,0,2,0,384,1
3,70.0,9135,2003,1682,1700,0,3,0,544,1
4,70.0,7763,1962,931,1283,0,3,0,506,1


Predicting the sale price from test data

In [123]:
x_test=pd.DataFrame(test)
y_pred=reg.predict(x_test)

In [ ]:
y_pred